In [ ]:
# Load package
import glob
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pingouin as pg
import os,shutil
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Data match
# Batch obtain data from .npy environmental data files in a folder
def list_npy_files(root_dir, npy_list):
    # Use glob to find all .npy files in the directory and its subdirectories
    npy_files = glob.glob(os.path.join(root_dir, '.../julian/*.npy'), recursive=True)
    npy_list.extend(npy_files)

# Initialize an empty list
list_npy = []

# Store result in list_csv
list_npy_files(r"...:\Environmental_factors_file_path", list_npy) #Modified to the path where environmental factors are stored 

# Read the phenology records
df = pd.read_hdf('..../pts.h5', key='data')
df = df.reset_index()

# Match phenological data and environmental factor data
for file in list_npy:
    fpath,fname=os.path.split(file)
    fname = fname.split('.')[0]
    env = np.load(file)
    # Creat DataFrame
    env=pd.DataFrame(env)
    # Calculate the average environmental factors between 0 and 6 months
    env2 = np.cumsum(env, axis=1) / (np.arange(7)+1)
    # Add the specified column name to the column
    columns = [fname+'_{}'.format(i) for i in range(0, 7)]
    env2.columns = columns
    df = df.join(env2)

# Adding the column order_cluster facilitates subsequent analysis
df = df.assign(species_cluster=df['species'].astype(str) + '_' + df['cate'].astype(str))


In [ ]:
# Using linear regression to remove annual trends
def remove_annual_trends(data, column):
    X = data['year'].astype('float64')
    y = data[column].astype('float64')
    model = OLS(y, sm.add_constant(X))
    results = model.fit()
    y_residual = results.resid
    return y_residual


# Optimal Pre-Season Selection
# Using linear regression to remove annual trends
def calculate_most_influential_column(factor, filtered_df1):
    # Create a dictionary to store the partial correlation coefficients for each factor column
    partial_corr = {}
    # Calculate the partial correlation coefficient for each factor column
    for i in range(7):
        column_name = factor + str(i)
        filtered_df1 =  filtered_df1.dropna(subset=[column_name])
        # Remove annual trends
        y_residual = remove_annual_trends(filtered_df1,column_name)
        # Calculate the partial correlation coefficient between the residuals of the factor column and the LUD/IOD column
        partial_corr[column_name] = pearsonr(y_residual, filtered_df1["IOD"])[0] #LUD/IOD are vegetation and insect phenology dates respectively

    # Convert the partial correlation coefficients to a Series and sort it (based on absolute values)
    partial_corr_series = pd.Series(partial_corr)
    # Preserve the non-absolute values
    
    partial_corr_series = partial_corr_series.abs().sort_values(ascending=False)
    
    # The most correlated (the column name with the highest absolute value of the partial correlation coefficient)
    most_influential_column = partial_corr_series.idxmax()

    return most_influential_column

In [ ]:
# Partial Correlation

# Environmental factors list
column_list =["fg_","hu_","rr_","qq_","tg_","total_evaporation_sum_","volumetric_soil_water_layer_1_",'soil_temperature_level_1_']

# Geographic Zoning
for s in range(0,1): 
    mean_r=[]  
    mean_r2=[] 
    for lon in range(-15,35,1):
        for lat in range(30,75,1):
            # Select the phenology data of the corresponding area
            filtered_df = df.query(f'{lon+s} <= Longitude <= {lon+1+s} and {lat} <= Latitude <= {lat+1}').reset_index() 
            filtered_df = (
                    filtered_df.groupby('species_cluster')
                      .filter(lambda g: len(g) > 30 and g['year'].nunique() > 10)
                )
            number_of_categories = (filtered_df.groupby('species_cluster')
                                    .size()
                                    .reset_index(name='count')
                                    .query('count > 30')
                                    .shape[0])
            
            if number_of_categories>=1:
                #Iterate over each phenological pattern
                all_list =[]
                species_list = filtered_df['species_cluster'].unique()
                scaler = StandardScaler()
                for sp in species_list:
                    MIC_list=[] 
                    filtered_df1=filtered_df[filtered_df["species_cluster"]==sp].copy()
                    if(filtered_df1['year'].nunique()>=10): 
                        for factor in column_list:  
                            MIC = calculate_most_influential_column(factor, filtered_df1)
                            MIC_list.append(MIC)#Record the optimal pre-season period for each environmental factor 
                        filtered_df1 =  filtered_df1.dropna(subset=MIC_list)
                        for fc in MIC_list:
                            # Remove annual trends
                            y_residual = remove_annual_trends(filtered_df1,fc)
                            # Standardize residuals
                            filtered_df1[fc] = scaler.fit_transform(y_residual.values.reshape(-1, 1)) 
                        pc_list =[]
                        # Calculate partial correlation
                        for factor in MIC_list:
                            covar_factors = MIC_list.copy()
                            covar_factors.remove(factor)
                            partial_corr = pg.partial_corr(data=filtered_df1, x='LUD', y=factor, covar=covar_factors) #IOD IOD
                            # Record the results
                            pc_list.append([partial_corr['r'][0],partial_corr['p-val'][0]])
                        all_list.append([sp,filtered_df1["order"].unique()[0],pc_list])
                
                # Organize data
                df_all = pd.DataFrame(all_list, columns=['sp', 'order', 'params'])
                for i, fa in enumerate(column_list):
                    df_all[[fa + 'r', fa + 'p']] = pd.DataFrame(df_all['params'].tolist(), index=df_all.index)[i].tolist()
                # Calculate the average partial correlation coefficients for a single geographical grid
                columns_to_average = [f"{fa}r" for fa in column_list]
                mean_r.append([lon + 0.5 + s, lat + 0.5, df_all[columns_to_average].mean().tolist()])
                # Calculate the average partial correlation coefficients (significant P<0.05) for a single geographical grid
                mean_r2.append([lon + 0.5 + s, lat + 0.5, [df_all[df_all[f"{fa}p"] < 0.05][f"{fa}r"].mean() for fa in column_list]])
    
    # Organize and output data
    all_list_grid = [item[:2] + item[2] for item in mean_r]  
    columns = ["lon","lat","fg_r","hu_r","rr_r","qq_r","tg_r","total_evaporation_sum_r","volumetric_soil_water_layer_1_r",'soil_temperature_level_1_r']
    all_list_grid = pd.DataFrame(all_list_grid, columns=columns)
    all_list_grid.to_csv("...\\IOD_PC_n" + str(s) + ".csv", index=False) 


In [ ]:
# Ridge Regression

# Environmental factors list
column_list =["fg_","hu_","rr_","qq_","tg_","total_evaporation_sum_","volumetric_soil_water_layer_1_",'soil_temperature_level_1_']

# Geographic Zoning
for s in range(0,1): 
    mean_r=[]  
    for lon in range(-15,35,1):
        for lat in range(30,75,1):
            # Select the phenology data of the corresponding area
            filtered_df = df.query(f'{lon+s} <= Longitude <= {lon+1+s} and {lat} <= Latitude <= {lat+1}').reset_index()  
            filtered_df = (
                    filtered_df.groupby('species_cluster')
                      .filter(lambda g: len(g) > 30 and g['year'].nunique() > 10)
                )
            number_of_categories = (filtered_df.groupby('species_cluster')
                                    .size()
                                    .reset_index(name='count')
                                    .query('count > 30')
                                    .shape[0])
            if number_of_categories>=1:
                #Iterate over each phenological pattern
                all_list =[]
                species_list = filtered_df['species_cluster'].unique()
                scaler = StandardScaler()
                for sp in species_list:
                    MIC_list=[] 
                    filtered_df1=filtered_df[filtered_df["species_cluster"]==sp].copy()
                    if(filtered_df1['year'].nunique()>=10): 
                        for factor in column_list:  
                            MIC = calculate_most_influential_column(factor, filtered_df1)
                            MIC_list.append(MIC)#Record the optimal pre-season period for each environmental factor 
                        filtered_df1 =  filtered_df1.dropna(subset=MIC_list)
                        for fc in MIC_list:
                            # Remove annual trends
                            y_residual = remove_annual_trends(filtered_df1,fc)
                            # Standardize residuals
                            filtered_df1[fc] = scaler.fit_transform(y_residual.values.reshape(-1, 1)) 
                    
                        # Ridge Regression
                        X = filtered_df1[MIC_list]
                        y = filtered_df1['LUD'] #IOD/IOD
                        ridge_model = Ridge(alpha=1.0)
                        ridge_model.fit(X, y)
                        # Records the results
                        pc_list = []
                        coefficients = ridge_model.coef_[0:len(MIC_list)]
                        for coeff in coefficients:
                            pc_list.append([coeff])
                        all_list.append([sp,filtered_df1["order"].unique()[0],pc_list])
                
                # Organize data
                df_all = pd.DataFrame(all_list, columns=['sp','order','params']) 
                for i, fa in enumerate(column_list):
                    df_all[fa + "r"] = df_all['params'].apply(lambda x: x[i][0])

                # Calculate the average Ridge Regression correlation coefficients for a single geographical grid
                columns_to_average = [fa + "r" for fa in column_list]
                mean_r.append([lon +0.5, lat + 0.5+ s, df_all[columns_to_average].mean().tolist()])
    
    #Organize and output data
    all_list_grid = [item[:2] + item[2] for item in mean_r]  
    columns = ["lon","lat","fg_r","hu_r","rr_r","qq_r","tg_r","total_evaporation_sum_r","volumetric_soil_water_layer_1_r",'soil_temperature_level_1_r']
    all_list_grid = pd.DataFrame(all_list_grid, columns=columns)
    all_list_grid.to_csv("...\\LUD_RC_n" + str(s) + ".csv", index=False)
    

In [ ]:
# Random forest

# Environmental factors list
column_list =["fg_","hu_","rr_","qq_","tg_","total_evaporation_sum_","volumetric_soil_water_layer_1_",'soil_temperature_level_1_']

# Geographic Zoning
for s in range(0,1): 
    mean_r=[]  
    for lon in range(-15,35,1):
        for lat in range(30,75,1):
            # Select the phenology data of the corresponding area
            filtered_df = df.query(f'{lon+s} <= Longitude <= {lon+1+s} and {lat} <= Latitude <= {lat+1}').reset_index()  
            filtered_df = (
                    filtered_df.groupby('species_cluster')
                      .filter(lambda g: len(g) > 30 and g['year'].nunique() > 10)
                )
            number_of_categories = (filtered_df.groupby('species_cluster')
                                    .size()
                                    .reset_index(name='count')
                                    .query('count > 30')
                                    .shape[0])
            if number_of_categories>=1:
                #Iterate over each phenological pattern
                all_list =[]
                species_list = filtered_df['species_cluster'].unique()
                scaler = StandardScaler()
                for sp in species_list:
                    MIC_list=[] 
                    filtered_df1=filtered_df[filtered_df["species_cluster"]==sp].copy()
                    if(filtered_df1['year'].nunique()>=10): 
                        for factor in column_list:  
                            MIC = calculate_most_influential_column(factor, filtered_df1)
                            MIC_list.append(MIC) #Record the optimal pre-season period for each environmental factor 
                        filtered_df1 =  filtered_df1.dropna(subset=MIC_list)
                        for fc in MIC_list:
                            # Remove annual trends
                            y_residual = remove_annual_trends(filtered_df1,fc)
                            # Standardize residuals
                            filtered_df1[fc] = scaler.fit_transform(y_residual.values.reshape(-1, 1)) 
                        
                        # Random forest
                        pc_list =[]
                        X = filtered_df1[MIC_list]
                        y = filtered_df1['LUD'] #IOD/IOD
                        rf_model = RandomForestRegressor(n_estimators=100)
                        rf_model.fit(X, y)
                        feature_importances = rf_model.feature_importances_
                        importances = feature_importances[:len(MIC_list)]
                        pc_list = [[importance] for importance in importances]
                        all_list.append([sp, filtered_df1["order"].unique()[0], pc_list])
                
                # Organize data
                df_all = pd.DataFrame(all_list, columns=['sp','order','params']) 
                for i, fa in enumerate(column_list):
                    df_all[fa + "r"] = df_all['params'].apply(lambda x: x[i][0])
                # Calculate the average random forest feature importance for a single geographical grid
                columns_to_average = [fa + "r" for fa in column_list]
                mean_r.append([lon + 0.5 + s, lat + 0.5, df_all[columns_to_average].mean().tolist()])
    
    #Organize and output data
    all_list_grid = [item[:2] + item[2] for item in mean_r]  
    columns = ["lon","lat","fg_r","hu_r","rr_r","qq_r","tg_r","total_evaporation_sum_r","volumetric_soil_water_layer_1_r",'soil_temperature_level_1_r']
    all_list_grid = pd.DataFrame(all_list_grid, columns=columns)
    all_list_grid.to_csv("...\\LUD_RF_r" + str(s) + ".csv", index=False) 